In [1]:
import cv2 as cv
import face_recognition
import pickle
import pandas as pd
from datetime import datetime, time
import tensorflow as tf 

In [2]:
# Load trained models
with open("face_recognition_model.pkl", "rb") as model_file:
    knn_model = pickle.load(model_file)

with open("label_encoder.pkl", "rb") as encoder_file:
    label_encoder = pickle.load(encoder_file)

In [3]:
# Load your trained emotion detection model
emotion_model = tf.keras.models.load_model("emotion_detection_model.h5")  # Update with your model's path
emotion_model.compile(
    optimizer="adam",  # Use the same optimizer as during training
    loss="categorical_crossentropy",  # Use the same loss function
    metrics=["accuracy"]
)
emotion_labels = ["happy", "sad", "neutral", "angry", "surprised","disgusted","fearful"]  # Update with your model's classes

In [ ]:
# Initialize attendance log
attendance = {}

# Time window for the attendance system
start_time = time(13, 40)
end_time = time(22, 58)

# Function to detect emotions using the trained model
def detect_emotion(face_img):
    # Resize face to the input size of your model (e.g., 48x48 or 64x64)
    face_resized = cv.resize(face_img, (48, 48))  # Update with your model's input size
    face_resized = cv.cvtColor(face_resized, cv.COLOR_BGR2GRAY)  # If your model expects grayscale
    face_resized = face_resized / 255.0  # Normalize pixel values
    face_resized = np.expand_dims(face_resized, axis=0)  # Add batch dimension
    face_resized = np.expand_dims(face_resized, axis=-1)  # Add channel dimension if needed

    # Predict emotion
    emotion_probs = emotion_model.predict(face_resized)
    emotion_index = np.argmax(emotion_probs)
    return emotion_labels[emotion_index]

# Check if within time window
def is_within_time_window(start, end):
    current_time = datetime.now().time()
    return start <= current_time <= end

# Start video capture
cap = cv.VideoCapture(0)

print("Attendance system is active between 1:30 AM and 2:42 AM.")

while True:
    # Check if within the time window
    if not is_within_time_window(start_time, end_time):
        print("Attendance system is inactive. Waiting for the next time window.")
        break

    success, img = cap.read()
    if not success:
        print("Failed to capture frame. Exiting.")
        break

    # Resize for faster processing
    imgS = cv.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv.cvtColor(imgS, cv.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(imgS)
    face_encodings = face_recognition.face_encodings(imgS, face_locations)

    for face_encoding, face_loc in zip(face_encodings, face_locations):
        # Predict student ID
        predictions = knn_model.kneighbors([face_encoding], n_neighbors=1)
        match_index = predictions[1][0][0]
        student_id = label_encoder.inverse_transform([match_index])[0]

        if student_id not in attendance:
            # Mark attendance as "Present"
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            face_img = img[y1:y2, x1:x2]

            # Detect emotion
            emotion = detect_emotion(face_img)

            # Update attendance
            attendance[student_id] = {
                "Time": datetime.now().strftime("%H:%M:%S"),
                "Emotion": emotion,
                "Status": "Present"
            }

            # Draw bounding box and label
            cv.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(img, f"{student_id} ({emotion})", (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

    # Display the video
    cv.imshow("Attendance System", img)

    # Quit with 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

# Check for any students who didn't get marked as "Present"
for student_id in label_encoder.classes_:
    if student_id not in attendance:
        attendance[student_id] = {
            "Time": "Not detected",
            "Emotion": "N/A",
            "Status": "Absent"
        }

# Save attendance to CSV
attendance_df = pd.DataFrame.from_dict(attendance, orient="index")
attendance_df.index.name = "Student ID"
attendance_df.to_csv("Attendance.csv")
print("Attendance saved to Attendance.csv.")


Attendance system is active between 1:30 AM and 2:42 AM.
